In [4]:
import glob
import json
import pandas
import plotly as py
import functools
import pprint
import numpy as np
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

DATA_PATH = "C:/Users/utkar/Downloads/large_data.json" # Change this for data
ZIP_CODE_DATA_PATH = "C:/Users/utkar/Downloads/zipcode_locations.csv" # Change this for zipcode
pprinter = pprint.PrettyPrinter(indent = 4)

This notebook converts the merged restaurant data into one feature file. Each column is a machine learning type, i.e one of the following four:
Nominal
Ordinal
Interval
Ratio

To generate a features.csv for yourself, do the following:  
1- Download the merged data from the following link:
https://drive.google.com/drive/u/1/folders/1KdMxGtI8dZmoNDhl0Vsd8h1N1cocQdiU

2- Download the zipcode dataset from the following link:
https://drive.google.com/drive/u/1/folders/1KdMxGtI8dZmoNDhl0Vsd8h1N1cocQdiU

3- Change the DATA_PATH and ZIP_CODE_DATA_PATH variable in the cell defined above to the path of your downloaded data from step 1 & 2.

4- Run all cells one after the other.

Each cell describes what it does.

In [5]:
# Load Zipcode Data
zipcodes = {}

with open(ZIP_CODE_DATA_PATH, 'r') as inputfile:
    header = inputfile.readline().replace("\"", "").split(',')
    
    for line in inputfile.readlines():
        vals = line.replace("\"", "").split(',')
        obj = {}
        zipcode = None
        for key, val in zip(header, vals):
            obj[key.strip()] = val.strip()
            if key.strip() == 'Zipcode':
                zipcode = int(val)
                
        zipcodes[zipcode] = obj

pprinter.pprint(zipcodes[90008])
# print(zipcodes["90008"])    

{   'City': 'LOS ANGELES',
    'Decommisioned': 'false',
    'EstimatedPopulation': '21174',
    'Lat': '34.00',
    'Location': 'NA-US-CA-LOS ANGELES',
    'LocationType': 'PRIMARY',
    'Long': '-118.34',
    'State': 'CA',
    'TaxReturnsFiled': '13333',
    'TotalWages': '426689767',
    'ZipCodeType': 'STANDARD',
    'Zipcode': '90008'}


In [7]:
# Load all data
all_restaurants = []
with open(DATA_PATH, "r") as inputfile:
    for line in inputfile.readlines():
    #for i in range(100):
        #line = inputfile.readline()
        all_restaurants.append(json.loads(line))

pprinter.pprint("total:" + str(len(all_restaurants)))
pprinter.pprint("restaurant_attributes:"+ str(all_restaurants[0].keys()))

def merge_sets(x, y):
    for item in y:
        x.add(item)
    return x

def get_keys(x):
    if x:
        return list(x.keys())
    return []
all_google_keys = functools.reduce(lambda x,y: merge_sets(x,y),map(lambda x: get_keys(x.get("google")), all_restaurants), set())
all_uigoogle_keys = functools.reduce(lambda x,y: merge_sets(x,y),map(lambda x: get_keys(x.get("uigoogle")), all_restaurants), set())
all_uiyelp_keys = functools.reduce(lambda x,y: merge_sets(x,y),map(lambda x: get_keys(x.get("uiyelp")), all_restaurants), set())
all_yelp_keys = functools.reduce(lambda x,y: merge_sets(x,y),map(lambda x: get_keys(x.get("yelp")), all_restaurants), set())

pprinter.pprint("All google attributes: ")
pprinter.pprint(str(all_google_keys))
pprinter.pprint("All uigoogle attributes: ")
pprinter.pprint(str(all_uigoogle_keys))
pprinter.pprint("All uiyelp attributes: ")
pprinter.pprint(str(all_uiyelp_keys))
pprinter.pprint("All yelp attributes: ")
pprinter.pprint(str(all_yelp_keys))


'total:9100'
"restaurant_attributes:dict_keys(['uigoogle', 'google', 'uiyelp', 'yelp'])"
'All google attributes: '
("{'photos', 'plus_code', 'opening_hours', 'scope', 'url', 'id', "
 "'permanently_closed', 'rating', 'icon', 'reference', 'place_id', "
 "'international_phone_number', 'adr_address', 'vicinity', "
 "'formatted_phone_number', 'geometry', 'types', 'formatted_address', "
 "'address_components', 'name', 'reviews', 'html_attributions', 'price_level', "
 "'utc_offset', 'website'}")
'All uigoogle attributes: '
"{'popularTimes', 'place_id', 'amenities', 'name', 'url', 'reviews'}"
'All uiyelp attributes: '
"{'amenities', 'name', 'phone', 'reviews', 'id'}"
'All yelp attributes: '
("{'categories', 'rating', 'distance', 'coordinates', 'price', 'review_count', "
 "'transactions', 'image_url', 'display_phone', 'phone', 'name', 'url', "
 "'location', 'is_closed', 'id', 'alias'}")


In [6]:
# All the possible amenities that there can be
all_amenities = set([])
amenities = map(lambda x: x["uiyelp"]["amenities"], all_restaurants)

for restaurant_amenities in amenities:
    for amenity in restaurant_amenities:
        all_amenities.add(amenity)
        
pprinter.pprint(all_amenities)

{   'Accepts Apple Pay',
    'Accepts Bitcoin',
    'Accepts Credit Cards',
    'Accepts Google Pay',
    'Alcohol',
    'Ambience',
    'Attire',
    'Best Nights',
    'Bike Parking',
    'Caters',
    'Coat Check',
    'Delivery',
    'Dogs Allowed',
    'Drive-Thru',
    'Gender Neutral Restrooms',
    'Good For',
    'Good For Dancing',
    'Good for Groups',
    'Good for Kids',
    'Good for Working',
    'Happy Hour',
    'Has Dairy-free Options',
    'Has Gluten-free Options',
    'Has Halal Options',
    'Has Kosher Options',
    'Has Pool Table',
    'Has Soy-free Options',
    'Has TV',
    'Liked by Vegans',
    'Liked by Vegetarians',
    'Music',
    'Noise Level',
    'Offers Military Discount',
    'Outdoor Seating',
    'Parking',
    'Smoking',
    'Take-out',
    'Takes Reservations',
    'Waiter Service',
    'Wheelchair Accessible',
    'Wi-Fi'}


In [7]:
# All possible cuisines
all_cuisines = set([])
cuisines = map(lambda x: x["yelp"]["categories"], all_restaurants)

for restaurant_cuisines in cuisines:
    for cuisine in restaurant_cuisines:
        all_cuisines.add(cuisine['alias'])
        
pprinter.pprint(all_cuisines)

{   'armenian',
    'asianfusion',
    'bars',
    'bbq',
    'beer_and_wine',
    'beerbar',
    'breakfast_brunch',
    'british',
    'bubbletea',
    'buffets',
    'burgers',
    'cafes',
    'cajun',
    'chicken_wings',
    'chickenshop',
    'chinese',
    'cocktailbars',
    'coffee',
    'convenience',
    'cuban',
    'delis',
    'desserts',
    'diners',
    'eventplanning',
    'filipino',
    'foodstands',
    'foodtrucks',
    'gastropubs',
    'greek',
    'hawaiian',
    'hotdogs',
    'hotpot',
    'indpak',
    'italian',
    'japanese',
    'latin',
    'lounges',
    'mediterranean',
    'mexican',
    'mideastern',
    'mongolian',
    'newamerican',
    'nicaraguan',
    'noodles',
    'organic_stores',
    'persian',
    'pizza',
    'poke',
    'pubs',
    'salad',
    'salvadoran',
    'sandwiches',
    'seafood',
    'seafoodmarkets',
    'sportsbars',
    'steak',
    'sushi',
    'tacos',
    'taiwanese',
    'tapas',
    'thai',
    'tradamerican',
    'v

In [8]:
# Split multi value fields
# Parking
# Good for
# Ambience
unique_parking_values = set([])
unique_good_for_values = set([])
unique_ambience_values = set([])
parking_vals = map(lambda x: x["uiyelp"]['amenities'].get('Parking') if x["uiyelp"]['amenities'].get('Parking') else [], all_restaurants)
good_for_vals = map(lambda x: x["uiyelp"]['amenities']['Good For'] if x["uiyelp"]['amenities'].get('Good For') else [], all_restaurants)
ambience_vals = map(lambda x: x["uiyelp"]['amenities']['Ambience'] if x["uiyelp"]['amenities'].get('Ambience') else [], all_restaurants)

for val in parking_vals:
    if val != []:
        for split_val in val.split(','):
            unique_parking_values.add(split_val.strip())
        
for val in good_for_vals:
    if val != []:
        for split_val in val.split(','):
            unique_good_for_values.add(split_val.strip())
        
for val in ambience_vals:
    if val != []:
        for split_val in val.split(','):
            unique_ambience_values.add(split_val.strip())
            
pprinter.pprint(unique_parking_values)
pprinter.pprint(unique_good_for_values)
pprinter.pprint(unique_ambience_values)

{'Street', 'Private Lot', 'Valet'}
{'Brunch', 'Late Night', 'Dinner', 'Breakfast', 'Dessert', 'Lunch'}
{'Casual', 'Intimate', 'Hipster', 'Trendy', 'Classy', 'Romantic', 'Divey'}


In [63]:
restaurants_df = {}
# Init the dataframe
for i, restaurant in enumerate(all_restaurants):
    restaurants_df[i] = {}
        
        
# Review rating put into DF dict
for i, restaurant in enumerate(all_restaurants):
    restaurants_df[i]['yelp_rating'] = float(restaurant["yelp"].get('rating')) if restaurant["yelp"].get('rating') else None
    restaurants_df[i]['google_rating'] = float(restaurant["google"].get('rating')) if restaurant["google"].get('rating') else None
    
# Amenities put into DF dict
for i, restaurant in enumerate(all_restaurants):
    for amenity in restaurant["uiyelp"]['amenities']:
        restaurants_df[i][amenity] = restaurant["uiyelp"]['amenities'][amenity]

# Cuisines put into DF dict
for i, restaurant in enumerate(all_restaurants):
#     print(restaurants_yelp[key]['categories'])
    res_cuisines = map(lambda x: x['alias'], restaurant['yelp']['categories'])
    for cuisine in res_cuisines:
        restaurants_df[i][cuisine] = 'yes'
        

# Split multi nominal attributes by splitting them up
for i, restaurant in enumerate(all_restaurants):
#     print(restaurant["uiyelp"]["amenities"].get('Parking'))
    if restaurant["uiyelp"]["amenities"].get('Parking'):
        for parking in restaurant["uiyelp"]["amenities"]['Parking'].split(","):
            restaurants_df[i]['parking_' + parking.strip()] = "yes"
            
    if restaurant["uiyelp"]["amenities"].get('Ambience'):
        for ambience in restaurant["uiyelp"]["amenities"]['Ambience'].split(","):
            restaurants_df[i]['ambience_' + ambience.strip()] = "yes"
        
    if restaurant["uiyelp"]["amenities"].get('Good For'):
        for good_for in restaurant["uiyelp"]["amenities"]['Good For'].split(","):
            restaurants_df[i]['good_for_' + good_for.strip()] = "yes"

# Review count put into DF dict
for i, restaurant in enumerate(all_restaurants):
    restaurants_df[i]['review_count'] = restaurant["yelp"].get('review_count')
    
# Put price into DF dict
for i, restaurant in enumerate(all_restaurants):
#     print(restaurants_yelp[key].get('price'))
    restaurants_df[i]['yelp_price'] = len(restaurant['yelp'].get('price')) if restaurant['yelp'].get('price') else None
    restaurants_df[i]['google_price'] = restaurant['google'].get('price_level')
    
# restaurants_df[list(restaurants_df.keys())[0]].keys()
# Put location into the DF
locations = map(lambda x: (x['google']['geometry']['location']['lat'], x['google']['geometry']['location']['lng']), all_restaurants)
for i, location in enumerate(locations):
    restaurants_df[i]['location'] = location

# Parse Popular times data here
import re
regex = r'[0-9]*%'
for i,restaurant in enumerate(all_restaurants):
    if 'uigoogle' in restaurant.keys():
        if restaurant['uigoogle'].get('popularTimes') and len(restaurant['uigoogle'].get('popularTimes')) > 0:
            time = restaurant['uigoogle']['popularTimes']
            for day in time:
                total = 0.0
                count = 0.0
                for business in time[day]:
                    match = re.match(regex, business)
                    if match != None:
                        business = match.group(0)
                        business = business.replace('%','')

                        if business != 0 and len(business) > 0:
                            total += int(business)
                            count += 1

                if count != 0:
                    restaurants_df[i][day] = total/count
        
                
# Add zipcode data to the instance
for i,restaurant in enumerate(all_restaurants):
    addr_components = restaurant['google']['address_components']
    zipcode = None
    for comp in addr_components:
        if 'postal_code' in comp['types']:
            zipcode = comp['long_name']
            break
    if zipcode != None:
        zipcode = zipcode.split('-')[0][:5]
        try:
            restaurants_df[i]['zipcode'] = zipcode
            zipcode = zipcodes[int(zipcode)]
            restaurants_df[i]['zipcode_TotalWages'] = zipcode['TotalWages']
            restaurants_df[i]['zipcode_EstimatedPopulation'] = zipcode['EstimatedPopulation']
            restaurants_df[i]['zipcode_TotalWages'] = zipcode['TotalWages']
        except Exception as E:
            restaurants_df[i]['zipcode'] = None
            restaurants_df[i]['zipcode_TotalWages'] = None
            restaurants_df[i]['zipcode_EstimatedPopulation'] = None
            restaurants_df[i]['zipcode_TotalWages'] = None

    
pprinter.pprint(restaurants_df[20])

{   'Accepts Apple Pay': 'No',
    'Accepts Credit Cards': 'Yes',
    'Accepts Google Pay': 'No',
    'Alcohol': 'No',
    'Ambience': 'Casual',
    'Attire': 'Casual',
    'Bike Parking': 'Yes',
    'Caters': 'No',
    'Delivery': 'No',
    'Fridays': 43.166666666666664,
    'Gender Neutral Restrooms': 'Yes',
    'Good for Groups': 'Yes',
    'Good for Kids': 'Yes',
    'Has TV': 'No',
    'Mondays': 35.833333333333336,
    'Noise Level': 'Average',
    'Outdoor Seating': 'No',
    'Saturdays': 33.708333333333336,
    'Sundays': 26.956521739130434,
    'Take-out': 'Yes',
    'Takes Reservations': 'No',
    'Thursdays': 42.791666666666664,
    'Tuesdays': 47.833333333333336,
    'Wednesdays': 33.958333333333336,
    'Wi-Fi': 'No',
    'ambience_Casual': 'yes',
    'google_price': 1,
    'google_rating': 3.9,
    'location': (34.13624959999999, -117.87174170000003),
    'mexican': 'yes',
    'review_count': 47,
    'yelp_price': 1,
    'yelp_rating': 2.5,
    'zipcode': '91740',
    'zi

In [ ]:
# Nearest neighbors for geo localized features
# Create nearest neighbors to get those features
RADIUS = 1.0 # Miles 
RATING_FIELD = 'yelp_rating'
PRICE_FIELD = 'yelp_price'

from math import radians, cos, sin, asin, sqrt

def haversine(x, y):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1 = x[1], x[0]
    lon2, lat2 = y[1], y[0]
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 3956 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

X = []
all_nearest = {}

for key1 in restaurants_df:
    nearest = []
    for key2 in restaurants_df:
        if key1 != key2:
            if haversine(restaurants_df[key1]['location'], restaurants_df[key2]['location']) <= RADIUS:
                nearest.append(key2)
                
    all_nearest[key1] = nearest

print(max(all_nearest, key=lambda x: len(all_nearest[x])))
# Avg rating of nearby restaurants
for key in all_nearest:
    total_rating = 0.0
    total_count = 0.0
    for nearest in all_nearest[key]:
        near_res = restaurants_df[nearest]
        if near_res[RATING_FIELD] != None:
            total_rating += near_res[RATING_FIELD]
            total_count += 1
        
    restaurants_df[key]['near_res_avg_rating'] = total_rating / total_count if total_count != 0 else None

# Avg price of nearby restaurants
for key in all_nearest:
    total_price = 0.0
    total_count = 0.0
    for nearest in all_nearest[key]:
        near_res = restaurants_df[nearest]
        if near_res[PRICE_FIELD] != None:
            total_rating += near_res[PRICE_FIELD]
            total_count += 1
        
    restaurants_df[key]['near_res_avg_price'] = total_price / total_count if total_count != 0 else None
    
# Avg rating of nearby cuisines
for cuisine in all_cuisines:
    for key in all_nearest:
        cuisine_count = 0
        cuisine_rating_total = 0.0
        cuisine_price_total = 0.0
        for nearest in all_nearest[key]:
            near_res = restaurants_df[nearest]
            if cuisine in near_res:
                if near_res[cuisine] == 'yes' and near_res.get(PRICE_FIELD):
                    cuisine_count += 1
                    cuisine_rating_total += near_res[RATING_FIELD]
                    cuisine_price_total += near_res[PRICE_FIELD]
                    
        restaurants_df[key]['near_' + cuisine + '_count'] = cuisine_count
        restaurants_df[key]['near_' + cuisine + '_avg_rating'] = cuisine_rating_total/cuisine_count if cuisine_count != 0 else None
        restaurants_df[key]['near_' + cuisine + '_avg_price'] = cuisine_price_total/cuisine_count if cuisine_count != 0 else None
# for nearest in neighbors[1]:
#     print(len(nearest))

# Populate popular times of restaurants with avg of nearby
for key1 in restaurants_df:
    days = ["Sundays", "Mondays", "Tuesdays", "Wednesdays",
               "Thursdays", "Fridays", "Saturdays"]
    restaurant = restaurants_df[key1]
    
    for day in days:
        if not restaurant.get(day):
            day_total = 0.0
            day_count = 0.0
            for key2 in all_nearest[key1]:
                near_res = restaurants_df[key2]
                if near_res.get(day):
                    day_total += near_res[day]
                    day_count += 1
            restaurant[day] = day_total / day_count if day_count > 0 else None
            
pprinter.pprint(restaurants_df[10])

In [ ]:
# parse location in lat long
for key in restaurants_df:
    location = restaurants_df[key]['location']
    restaurants_df[key]['lat'] = location[0]
    restaurants_df[key]['long'] = location[1]

In [15]:
# Create dataframe from dict
dataframe = pandas.DataFrame(data = list(restaurants_df.values()))
pprinter.pprint(list(dataframe.columns))
# Fill empty values with None
# You can add your own logic here to fill with other types of 'missing values'
for column in dataframe.columns:
    dataframe[column] = dataframe[column].fillna("None")

[   'Accepts Apple Pay',
    'Accepts Bitcoin',
    'Accepts Credit Cards',
    'Accepts Google Pay',
    'Accepts Insurance',
    'Ages Allowed',
    'Alcohol',
    'Ambience',
    'Attire',
    'Best Nights',
    'Bike Parking',
    'Cannabis Products',
    'Caters',
    'Coat Check',
    'Delivery',
    'Dogs Allowed',
    'Drive-Thru',
    'Fridays',
    'Gender Neutral Restrooms',
    'Good For',
    'Good For Dancing',
    'Good for Groups',
    'Good for Kids',
    'Good for Working',
    'Happy Hour',
    'Has Dairy-free Options',
    'Has Gluten-free Options',
    'Has Halal Options',
    'Has Kosher Options',
    'Has Pool Table',
    'Has Soy-free Options',
    'Has TV',
    'Liked by Vegans',
    'Liked by Vegetarians',
    'Mondays',
    'Music',
    'Noise Level',
    'Offers Military Discount',
    'Open to All',
    'Outdoor Seating',
    'Parking',
    'Saturdays',
    'Smoking',
    'Sundays',
    'Take-out',
    'Takes Reservations',
    'Thursdays',
    'Tuesdays',


In [18]:
# Drop cuisines that have less than 100 'yes'
# You can skip this cell if you don't want to drop those cuisines.
for cuisine in all_cuisines:
#     print(dataframe[cuisine].value_counts())
    if len(dataframe[cuisine].value_counts()) <= 1 or dataframe[cuisine].value_counts()[1]< 100:
        dataframe = dataframe.drop(columns = cuisine)


In [64]:
# Remove restaurants with no popular times
# you can skip this cell if you don't want to drop those restaurants
days = ["Sundays", "Mondays", "Tuesdays", "Wednesdays",
               "Thursdays", "Fridays", "Saturdays"]

for day in days:
    dataframe = dataframe[dataframe[day] != None]

In [20]:
# You are done.
dataframe.to_csv('latest_features.csv', header = True, index = False)

In [ ]:

print(dataframe[dataframe['Fridays'] != -1])

In [65]:
import plotly.graph_objs as go
df = dataframe
df.head()

mapbox_access_token = 'pk.eyJ1IjoiemVlc2hhbmFiaWQ5NCIsImEiOiJjam9tMG5zc3kwNjhhM2twMThqdHNudml1In0.dsHqUf7IqNqCTJ-l9FbxXA'
# print(df.apply(lambda row: " ".join(list(map(str, filter(lambda x: row[x] == 'yes',
#                                                              list(all_cuisines))))), axis=1))
df['text'] = df.apply(lambda row: " ".join(list(map(str, filter(lambda x: row[x] == 'yes',
                                                                list(all_cuisines))))), axis=1)
scl = [ [0,"rgb(5, 10, 172)"],[0.35,"rgb(40, 60, 190)"],[0.5,"rgb(70, 100, 245)"],\
    [0.6,"rgb(90, 120, 245)"],[0.7,"rgb(106, 137, 247)"],[1,"rgb(220, 220, 220)"] ]

data = []

for i, cuisine in enumerate(all_cuisines):
    data.append(
        go.Scattermapbox(
        name=cuisine,
        lat=df[dataframe[cuisine] == 'yes']['lat'],
        lon=df[dataframe[cuisine] == 'yes']['long'],
        text=df[dataframe[cuisine] == 'yes']['text'],
        mode='markers',
        marker=dict(
            size=5,
            colorscale = scl,
            cmin = 0,
            color = i,
            cmax = len(all_cuisines)
        )
    )
)

layout = go.Layout(
    autosize=True,
    hovermode='closest',
    showlegend=True,
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=34,
            lon=-118
        ),
        pitch=0,
        zoom=5
    ),
)

fig = dict( data=data, layout=layout )
iplot( fig)

KeyError: ('caribbean', 'occurred at index 0')

In [21]:
POI = (34.0231339,-118.2794343)
nearest_POI = []
POI_zipcode = zipcodes[90007]
print(POI_zipcode)
for key in restaurants_df:
    restaurant = restaurants_df[key]
    
    if haversine(POI, restaurant['location']) <= RADIUS:
        nearest_POI.append(restaurant)
        


{'Zipcode': '90007', 'ZipCodeType': 'STANDARD', 'City': 'LOS ANGELES', 'State': 'CA', 'LocationType': 'PRIMARY', 'Lat': '34.02', 'Long': '-118.28', 'Location': 'NA-US-CA-LOS ANGELES', 'Decommisioned': 'false', 'TaxReturnsFiled': '12367', 'EstimatedPopulation': '20179', 'TotalWages': '262370773'}


In [54]:
from collections import Counter
print("Surrounding Restaurants Number:", len(nearest_POI))
surrounding_amenities = Counter()
surrounding_cuisines = Counter()
total_cuisines_avg = {}
total_cuisines_price = {}

for res in nearest_POI:
    for amenity in all_amenities:
        if res.get(amenity) and res[amenity] == 'Yes':
            if amenity in surrounding_amenities:
                surrounding_amenities[amenity] += 1
            else:
                surrounding_amenities[amenity] = 1
        
    for cuisine in all_cuisines:
        if res.get(cuisine) and res[cuisine] == 'yes':
            if cuisine in surrounding_cuisines:
                surrounding_cuisines[cuisine] += 1
            else:
                surrounding_cuisines[cuisine] = 1
             
            if res.get(RATING_FIELD):
                if cuisine in total_cuisines_avg:
                    total_cuisines_avg[cuisine][0] += res.get(RATING_FIELD)
                    total_cuisines_avg[cuisine][1] += 1
                else:
                    total_cuisines_avg[cuisine] = [res.get(RATING_FIELD), 1]
                
            if res.get(PRICE_FIELD):
                if cuisine in total_cuisines_price:
                    total_cuisines_price[cuisine][0] += res.get(PRICE_FIELD)
                    total_cuisines_price[cuisine][1] += 1
                else:
                    total_cuisines_price[cuisine] = [res.get(PRICE_FIELD), 1]
                
print("Surrounding Amenities", str(surrounding_amenities))
print("")
print("Surrounding Cuisines", str(surrounding_cuisines))
print("")

for key in total_cuisines:
    total_cuisines_avg[key] = total_cuisines_avg[key][0]/total_cuisines_avg[key][1]
    
for key in total_cuisines_price:
    total_cuisines_price[key] = total_cuisines_price[key][0]/total_cuisines_price[key][1]
    
    
print("")
print(total_cuisines_avg)
print("")
print(total_cuisines_price)

Surrounding Restaurants Number: 40
Surrounding Amenities Counter({'Take-out': 39, 'Good for Kids': 37, 'Good for Groups': 36, 'Bike Parking': 35, 'Accepts Credit Cards': 33, 'Outdoor Seating': 27, 'Caters': 21, 'Has TV': 20, 'Liked by Vegetarians': 17, 'Wheelchair Accessible': 16, 'Liked by Vegans': 12, 'Delivery': 11, 'Gender Neutral Restrooms': 9, 'Waiter Service': 7, 'Accepts Apple Pay': 6, 'Takes Reservations': 5, 'Accepts Google Pay': 5, 'Has Gluten-free Options': 5, 'Dogs Allowed': 5, 'Has Dairy-free Options': 4, 'Offers Military Discount': 3, 'Has Soy-free Options': 2, 'Has Halal Options': 2, 'Has Kosher Options': 1, 'Open to All': 1, 'Good for Working': 1})

Surrounding Cuisines Counter({'burgers': 6, 'mexican': 6, 'tradamerican': 5, 'sandwiches': 5, 'newamerican': 4, 'breakfast_brunch': 4, 'tacos': 4, 'hotdogs': 3, 'salad': 3, 'hotdog': 2, 'indpak': 2, 'seafood': 2, 'salvadoran': 2, 'juicebars': 2, 'beerbar': 2, 'foodtrucks': 2, 'pizza': 2, 'cheesesteaks': 1, 'himalayan': 1, '